# Report

## Policy-Based Methods
Whereas *Value-Based Methods* like Deep Q-Learning are obtaining an optimal policy $\pi_*$ by trying to estimate the optimal action-value function, *Policy-Based Methods* directly learn the optimal policy.  
Besides this simplification another advantage of a Policy-Based Method is the fact that it is able to handle either stochastic or continuous actions.  
On the one hand Policy-Based Methods are using the *Monte Carlo* (MC) approach for the estimate of expected return:

$ G_t = R_{t+1} + R_{t+2} + ... + R_T$, if the discount factor $\gamma=1$

As $G_t$ is estimated with the full trajectory this yields to a high *variance*, but to a low *bias*.  
On the other hand Value-Based Methods are using the *Temporal Difference* (TD) approach to estimate the return:

$ G_t = R_{t+1} + G_{t+1}$ , if $\gamma=1$

Here $G_{t+1}$ is the estimated total return an agent will obtain in the next state. As the estimate of $G_t$ is always depending on the estimate of the next state, the variance of these estimates is low but biased.  
The pros of both methods can be combined in one single algorithm namely the Actor-Critic Method.

## Actor-Critic Methods
In Actor-Critic Methods one uses two function approximators (usually neural networks) to learn a policy (Actor) and a value function (Critic). The process looks as follows:  

1) Observe state $s$ from environment and feed into the Actor.  
2) The output are action probabilities $\pi(a|s;\theta_\pi)$. Select one action stochastically and feed back to the environment.  
3) Observe next state $s'$ and reward $r$.  
4) Use the tuple $(s, a, r, s')$ for the TD estimate $y=r + \gamma V(s'; \theta_v)$  
5) Train the Critic by minimizing the loss $L=(y - V(s;\theta_v)^2$.  
6) Calculate the advantage $A(s,a) = r + \gamma V(s'; \theta_v) - V(s; \theta_v)$.  
7) Train the Actor using the advantage.

## Deep Deterministic Policy Gradient
The following section refers to [Lillicrap et al., 2016].  
Deep Deterministic Policy Gradient (DDPG) combines the Actor-Critic approach with Deep Q-Learning. The actor function $\mu(s;\theta_\mu)$ gives the current policy. It maps states to continuous deterministic actions. The critic $Q(s,a;\theta_q)$ on the other hand is used to calculate action values and is learned using the Bellman equation. DDPG is also using a *replay buffer* and *target networks* which already helped to improve performance for Deep Q-Learning. In a finite replay buffer tuples of $(s, a, r, s')$ are stored and then batches are sampled from this buffer to apply for network updates. This tackles the issue of correlated tuples arrised from sequentially exploring the environment. Target networks are used to decouple the TD target from the current action value when performing neutwork updates. The target network is a copy of the Actor and Critic Network which are used to calculated the target. One approach is to update the weights of the target networks $\theta'$ with the weights $\theta$ of the Actor and Critic network periodically. An other approach is to perform *soft updates*:

$ \theta' \leftarrow \tau \theta + (1-\tau)\theta'$ with $\tau \ll 1$

In order to scale features *batch normalization* is being applied. This normalizes each dimension across the samples of the minibatch. An other important issue is handling exploration. By adding a noise process $N$ an exploration policy $\mu'$ is constructed:

$\mu'(s_t) = \mu(s_t;\theta_{\mu,t})+N$

The DDPG process looks as follows:  
1) Observe state $s$ from environment and feed to Actor.  
2) Select action $a = \mu(s;\theta_\mu) + N$ and feed back to environment.  
3) Observe next state $s'$ and reward $r$.  
4) Store transition $(s, a, r, s')$ in replay buffer and sample random minibatch of $n$ tuples. Calculate the TD estimate 
$y = r + \gamma Q'(s', \mu'(s';\theta_\mu');\theta_q')$  
5) Train the Critic by minimizing the loss 
$L=\mathbb{E} \big[\big(y - Q(s,a;\theta_q)\big)^2\big]$  
6) Train Actor with policy gradient 
$\mathbb{E} \big[\nabla_{\theta_\mu} Q(s,a;\theta_q) | s=s_t, a=\mu(s_t;\theta_\mu) \big] = \mathbb{E} \big[\nabla_a Q(s,a;\theta_q)|s=s_t,a=\mu(s_t) \nabla_{\theta_\mu} \mu(s;\theta_\mu)|s=s_t\big] $  
7) Update both target networks using soft update

As one see, this is an off-policy algorithm because the policy which is evaluated uses action $a=\mu'(s';\theta_\mu')$. This is different from the policy which selects action $a = \mu(s;\theta_\mu) + N$.

## References
- Lillicrap, T., Hunt, J., Pritzel, A., Heess, N., Erez, T., Tassa, Y., Silver, D., and Wiestra, D., Continuous Control with Deep Reinforcement Learning, arXiv:1509.02971v5 [cs.LG] 29 Feb 2016